In [6]:
#6/1/2020: modified mono_st, st; deleted mono_st_new, st_new
#beware of whether the functions will alter the variables

#def lincom(<list of dictionaries>, <list of coeff>) e.g. lincom([u,v,w], [3,4,-1]) => return dictionary: 3u + 4v - w
def lincom(l, c):
    r = {}
    n = len(c) #len(l) should equal to len(c)
    for i in range(n):
        r = plus(r, scal(c[i], l[i]))
    return r

def show(d):
    for k in d:
        print(k, '------', d[k]) 

def plus(u, v): 
    result = v.copy()
    for k in u:
        if k in result:
            result[k] += u[k]
        else:
            result[k] = u[k]
    return result

def plus1(u, v): #v will CHANGE to u+v; u won't change
    for k in u:
        if k in v:
            v[k] += u[k]
        else:
            v[k] = u[k]
            
def scal(n, d): #d is a dictionary; return scalar product of d by n
    r = d.copy()
    for k in r:
        r[k] *= n
    return r

def scal1(n, d): #d will CHANGE
    for k in d:
        d[k] *= n
        
def concat(u, v): #u,v are dicts with xy-string/z-value-tuple keys; return uv
                    #watch out when u, v are empty dicts
    r = {}
    for k in u:
        for l in v:           
            r[k + l] = u[k] * v[l] #k + l joins two strings or tuples
    return r

def mono_sh(s, t): #s, t are xy-strings; return dictionary
    #inefficient; change it back to the original
    if s == '':
        return {t: 1}
    if t == '':
        return {s: 1}
    
    s1 = s[0 : len(s) - 1]
    t1 = t[0 : len(t) - 1]
    a = concat(mono_sh(s, t1), {t[-1]: 1})
    b = concat(mono_sh(s1, t), {s[-1]: 1})
    plus1(a, b) #b will change to a+b
    return b

def mono_st(head, s, t, dic): #head, s, t are TUPLES
    if s == (): #empty tuple
        r = head + t #join two tuples 
        if r in dic:
            dic[r] += 1
        else:
            dic[r] = 1
        return 
    
    if t == ():
        r = head + s
        if r in dic:
            dic[r] += 1
        else:
            dic[r] = 1
        return 
    
    heads = head + (s[0],)
    headt = head + (t[0],)
    headspt = head + (s[0]+t[0],)
    '''
    s_copy = list(s) #can't use s.copy()
    s.pop(0) #now s does not have the first number
    mono_st(heads, s, t, dic)
    t.pop(0) #pop also makes trouble. don't fucking know why
    mono_st(headt, s_copy, t, dic)
    mono_st(headspt, s, t, dic)
    '''
    mono_st(heads, s[1:len(s)], t, dic)
    mono_st(headt, s, t[1:len(t)], dic)
    mono_st(headspt, s[1:len(s)], t[1:len(t)], dic)
    
def mono_linst(head, s, t, dic): #linearized stuffle
    if s == (): #empty tuple
        r = head + t #join two tuples 
        if r in dic:
            dic[r] += 1
        else:
            dic[r] = 1
        return 
    
    if t == ():
        r = head + s
        if r in dic:
            dic[r] += 1
        else:
            dic[r] = 1
        return 
    
    heads = head + (s[0],)
    headt = head + (t[0],)
    
    mono_linst(heads, s[1:len(s)], t, dic)
    mono_linst(headt, s, t[1:len(t)], dic)    

def xysh(u, v): #u,v are dicts. But their keys are strings of xy-binary words
    result = {}
    for u_mono in u: #mono means monomial
        for v_mono in v:
            r = mono_sh(u_mono, v_mono)            
            scal1(u[u_mono]*v[v_mono], r)
            plus1(r, result)
    return result #the keys of result are also xy-binary words

def toxy(d): #d: dictionary whose keys are tuples. Transform d to another dictionary r whose keys are strings of xy-binary words
    r = {}
    for k in d:
        s = ''
        for n in k: #e.g. k = (2,3,9) then n is integer 2 -> 3 -> 9
            s += 'y'
            for i in range(n-1): #append 'y' with (n-1) 'x'
                s += 'x'
        r[s] = d[k]
    return r

def toz(d): #reverse of toxy.
    # 'yxxxyxyyyx': place of y: 0, 4, 6, 7, 8. Don't forget the last one: z2. 2 = len - 8 = 10 - 8
    r = {}
    for k in d: #e.g. k = 'yxxxyxyyyx'
        l = []
        for i in range(len(k)): #i=0 ~ len(k)-1
            if k[i] == 'y':
                l.append(i) #l records position of 'y'
                
        l.append(len(k)) #Don't forget the last one; now l = [0,4,6,7,8,10]
        ll = []
        i = 0
        while i <= len(l) - 2:
            ll.append(l[i+1] - l[i])
            i += 1
        
        t = tuple(ll)
        r[t] = d[k]
    return r

def sh(u, v): #u, v are dictionary whose keys are z-value tuples
    u1 = toxy(u)
    v1 = toxy(v)
    return toz(xysh(u1, v1)) #also return dictionary whose keys are z-value tuples
 
def st(u, v): #z-value stuffle
    result = {}
    for u_mono in u: 
        for v_mono in v:            
            r = {}; mono_st((), u_mono, v_mono, r) #r = u_mono st v_mono
            scal1(u[u_mono]*v[v_mono], r)
            plus1(r, result)
    return result

def linst(u, v): #z-value linearized stuffle
    result = {}
    for u_mono in u: 
        for v_mono in v:            
            r = {}; mono_linst((), u_mono, v_mono, r) #r = u_mono st v_mono
            scal1(u[u_mono]*v[v_mono], r)
            plus1(r, result)
    return result

def reg_sh(d):
    r = {}
    for k in d:
        if k[-1] != 1:
            #simply add k into r
            if k in r:
                r[k] += d[k]
            else:
                r[k] = d[k]
        else: 
            #add reg_sh(k) into r
            m = 0
            ones = []
            while k[-(m+1)] == 1:
                m += 1
                ones = ones + [1]
            k_list = list(k)
            k_len = len(k_list)
            w0 = k_list[0 : k_len - m] #k = w0 . {1}^m (' . ' is concatenation)
            w0[-1] -= 1 #w0 becomes w0'
            reg_k_1 = sh( {tuple(ones): (-1)**m}, {tuple(w0): 1} )
            
            reg_k = {}
            for key in reg_k_1: #concatenate reg_k with x at its end
                key_x = list(key); key_x[-1] += 1; key_x = tuple(key_x)
                reg_k[key_x] = reg_k_1[key]
                
            r = plus(r, scal(d[k],reg_k)) #rewrite with plus1, scal1          
    return r    

'''
import data

def mono_diff_del(n, s): 
    if s == 'x':
        return data.del_x[n]
    if s == 'y':             
        return scal(-1, data.del_x[n])
    
    s1 = s[0 : len(s) - 1]
    if s[-1] == 'x':
        a = concat(mono_diff_del(n, s1), {'x': 1})
        b = concat({s1: 1}, data.del_x[n])
        plus1(a, b) #b changes to a+b
        return b
    else: 
        a = concat(mono_diff_del(n, s1), {'y': 1})
        b = concat({s1: -1}, data.del_x[n])
        plus1(a, b)
        return b
'''
        
'''
def diff_del(n, d, delx): #the keys of d are xy-strings
    
    for k in d:
        k1 = k[0 : len(k)-1] #taking away the last character
        if k[-1] == 'x':
            a = concat(diff_del(n, k1, delx), {'x': 1})
            b = plus(a, concat(k1, delx[n]))
            plus1(r, b)
        else:          

'''
        
def xyDict_to_list(w, d): #the keys of d are xy-strings BELONGING TO H^0 and have to be of the SAME WEIGHT w = len(key)
    #create list with 2^(w-2) zeros
    l = []
    for i in range( 2**(w-2) ):
        l.append(0)
    
    for k in d:
        n = 0 #convert k to its index in l
        i = w-2 #second to last character of string k   
        while i >= 1: #traverse back till the second character
            if k[i] == 'y':
                n += 2**(w-2-i)
            i -= 1
        l[n] = d[k]
    return l
    

In [8]:
u = {
    (2,2): 1
}
v = {
    (3,5): 1
}
linst(u,v)

{(2, 2, 3, 5): 1,
 (2, 3, 2, 5): 1,
 (2, 3, 5, 2): 1,
 (3, 2, 2, 5): 1,
 (3, 2, 5, 2): 1,
 (3, 5, 2, 2): 1}